In [ ]:
# Import data
import pandas as pd
import sqlite3

# Define file path and table name
csv_file = 'NewbornNamesDK.csv'
table_name = 'baby_names'

# Connect to database
conn = sqlite3.connect('data.db')

# Read CSV data
df = pd.read_csv(csv_file, delimiter=';')

# Create table
conn.cursor().execute(f"DROP TABLE IF EXISTS {table_name}")

# Import data into table
df.to_sql(table_name, conn, index=False)
cursor = conn.cursor()

# Show schemas
schemas = cursor.execute("PRAGMA table_info('baby_names')").fetchall()
print(schemas)

print(f"Data from '{csv_file}' successfully imported into table '{table_name}'")


[(0, 'Year', 'INTEGER', 0, None, 0), (1, 'Name', 'TEXT', 0, None, 0), (2, 'Total', 'INTEGER', 0, None, 0), (3, 'Gender', 'TEXT', 0, None, 0)]
Data from 'NewbornNamesDK.csv' successfully imported into table 'baby_names'


![](https://drive.google.com/uc?export=view&id=1aawqW3t9VCmNJcz3weNsyfgUQt-3WEva)

#**Danish baby names analysis**
Being inspired by a case study from Datacamp, I decided to do this project as a fun activity with my friend who was expecting to have an angel in 2023.

* I used the data provided by Statistics Denmark about the names given to newborn children in 38 years (1985-2022) to understand how Danish baby name tastes changed by investigating trends of popularity.

* The data lists names, the gender and total number of babies born in each year. For processing speed purposes, I've limited the dataset to names which were given from 400 Danish babies in a given year.

#### ▶  **baby_names**
| Column | Type | Meaning |
| -------- | ------- | ------- |
| year | int | year |
| name | varchar | name |
| gender | varchar | gender of babies given *name* |
| total | int | number of babies of *gender* given *name* in that *year*|

##1. Classic Danish name:

* First, let's find top 6 classic Danish names that appeared most frequently through years.

In [ ]:
# Query
cursor.execute('SELECT name, sum(total), gender, count(year) as frequency FROM baby_names GROUP BY name ORDER BY count(year) DESC LIMIT 6')
# Fetch results
results = cursor.fetchall()
#Change the result into tabular format
from tabulate import tabulate
print(tabulate(results, headers=['Name', 'Total', 'Gender', 'Frequency'], tablefmt="fancy_grid"))



╒═══════════╤═════════╤══════════╤═════════════╕
│ Name      │   Total │ Gender   │   Frequency │
╞═══════════╪═════════╪══════════╪═════════════╡
│ Emil      │   20695 │ M        │          32 │
├───────────┼─────────┼──────────┼─────────────┤
│ Mikkel    │   22302 │ M        │          28 │
├───────────┼─────────┼──────────┼─────────────┤
│ Emma      │   19031 │ F        │          28 │
├───────────┼─────────┼──────────┼─────────────┤
│ Oliver    │   16961 │ M        │          27 │
├───────────┼─────────┼──────────┼─────────────┤
│ Mads      │   21669 │ M        │          26 │
├───────────┼─────────┼──────────┼─────────────┤
│ Christian │   25211 │ M        │          26 │
╘═══════════╧═════════╧══════════╧═════════════╛


##2. Popularity type:

* It seems like most of the classic names are male ones. Emma is holding her 2nd place. 👼

* Next, I would like to categorize the names ased on its popularity. Let's see whether the names fall into "classic" that appear in many years, or just "trendy" in a few years?

In [ ]:
cursor.execute('SELECT name, sum(total), CASE WHEN count(year) > 26 THEN "Classic" WHEN count(year) > 16 THEN "Semi-classic" WHEN count(year) > 10 THEN "Semi-trendy" ELSE "Trendy" END FROM baby_names GROUP BY name ORDER BY name')
results = cursor.fetchall()
print(tabulate(results, headers=['Name', 'Total', 'Popularity Type'], tablefmt="fancy_grid"))

╒═════════════╤═════════╤═══════════════════╕
│ Name        │   Total │ Popularity Type   │
╞═════════════╪═════════╪═══════════════════╡
│ Agnes       │     866 │ Trendy            │
├─────────────┼─────────┼───────────────────┤
│ Aksel       │    1317 │ Trendy            │
├─────────────┼─────────┼───────────────────┤
│ Alberte     │     412 │ Trendy            │
├─────────────┼─────────┼───────────────────┤
│ Alexander   │   10387 │ Semi-classic      │
├─────────────┼─────────┼───────────────────┤
│ Alfred      │    3282 │ Trendy            │
├─────────────┼─────────┼───────────────────┤
│ Alma        │    3187 │ Trendy            │
├─────────────┼─────────┼───────────────────┤
│ Amalie      │    5653 │ Semi-trendy       │
├─────────────┼─────────┼───────────────────┤
│ Anders      │   11720 │ Semi-trendy       │
├─────────────┼─────────┼───────────────────┤
│ Andreas     │   14050 │ Semi-classic      │
├─────────────┼─────────┼───────────────────┤
│ Anna        │   10981 │ Semi-cla

##3. Top-ranked female names since 1985
* Oh most of my friends' names fall into "Semi-classic". My cousin's name, Nanna, is "Trendy". It makes sense as she is way younger than me.

* Since there is only 1 classic female name in the first query, let's limit the search this gender and find out "What are the top-ranked female names since 1985?".

In [ ]:
cursor.execute('SELECT name, SUM(total), RANK() OVER (ORDER BY SUM(total) DESC) FROM baby_names WHERE gender = "F" GROUP BY name' )
results = cursor.fetchall()
print(tabulate(results, headers=['Name', 'Total', 'Female Name Rank'], tablefmt="fancy_grid"))

╒═══════════╤═════════╤════════════════════╕
│ Name      │   Total │   Female Name Rank │
╞═══════════╪═════════╪════════════════════╡
│ Emma      │   19031 │                  1 │
├───────────┼─────────┼────────────────────┤
│ Camilla   │   17206 │                  2 │
├───────────┼─────────┼────────────────────┤
│ Sofie     │   16724 │                  3 │
├───────────┼─────────┼────────────────────┤
│ Julie     │   16526 │                  4 │
├───────────┼─────────┼────────────────────┤
│ Ida       │   14155 │                  5 │
├───────────┼─────────┼────────────────────┤
│ Katrine   │   14098 │                  6 │
├───────────┼─────────┼────────────────────┤
│ Freja     │   12802 │                  7 │
├───────────┼─────────┼────────────────────┤
│ Laura     │   12474 │                  8 │
├───────────┼─────────┼────────────────────┤
│ Louise    │   12467 │                  9 │
├───────────┼─────────┼────────────────────┤
│ Cecilie   │   12302 │                 10 │
├─────────

##4. Picking a baby name
* This is the part for my beloved client (aka my friend who was the first person seeing this project).

* She is finding a female classic name ending with "ie" as it sounds lovely. It doesn't matter if it is top ranked or not. Ideally, it's become popular in the years since 2004.

In [ ]:
cursor.execute('SELECT name FROM baby_names WHERE gender = "F" AND year > 2004 AND name like "%ie" GROUP BY name ORDER BY SUM(total) DESC' )
results = cursor.fetchall()
print(tabulate(results, headers=['Name'], tablefmt="fancy_grid"))

╒═════════╕
│ Name    │
╞═════════╡
│ Sofie   │
├─────────┤
│ Julie   │
├─────────┤
│ Emilie  │
├─────────┤
│ Cecilie │
├─────────┤
│ Amalie  │
╘═════════╛


##5. How has "Sofie" name risen?
* Among 5 options, my friend was considering "Sofie" as her choice. It is the most popular female name ending with "ie" since 2004.

* Let's see how this name has risen over years.

In [ ]:
cursor.execute('SELECT year, name, total, SUM(total) OVER (ORDER BY year) FROM baby_names WHERE name = "Sofie" ORDER BY year' )
results = cursor.fetchall()
print(tabulate(results, headers=['Year', 'Name', 'Total', 'Cumulative Sofies'], tablefmt="fancy_grid"))

╒════════╤════════╤═════════╤═════════════════════╕
│   Year │ Name   │   Total │   Cumulative Sofies │
╞════════╪════════╪═════════╪═════════════════════╡
│   1990 │ Sofie  │     486 │                 486 │
├────────┼────────┼─────────┼─────────────────────┤
│   1991 │ Sofie  │     551 │                1037 │
├────────┼────────┼─────────┼─────────────────────┤
│   1992 │ Sofie  │     629 │                1666 │
├────────┼────────┼─────────┼─────────────────────┤
│   1993 │ Sofie  │     631 │                2297 │
├────────┼────────┼─────────┼─────────────────────┤
│   1994 │ Sofie  │     783 │                3080 │
├────────┼────────┼─────────┼─────────────────────┤
│   1995 │ Sofie  │     771 │                3851 │
├────────┼────────┼─────────┼─────────────────────┤
│   1996 │ Sofie  │     762 │                4613 │
├────────┼────────┼─────────┼─────────────────────┤
│   1997 │ Sofie  │     873 │                5486 │
├────────┼────────┼─────────┼─────────────────────┤
│   1998 │ S

##6. Top male names over years
* "Sofie" has had a steady rise and was popular from 1990 to 2012 as a "Semi-classic" name.

* Now, let's dive into male names. Among top classic Danish names, 5 out of 6 are male ones. They are classic but showing up in the dataset in most of the year doesn't necessarily mean that the timeless names were the most popular.

* Let's find out which name has been number one for the largest number of years and practice **SUBQUERY, CTE, and JOIN**.

In [ ]:
cursor.execute('WITH most_popular_male_name_by_year AS (SELECT b.year, name, total FROM baby_names b INNER JOIN (SELECT year, MAX(total) as max_num FROM baby_names WHERE gender = "M" GROUP BY year) as max_table ON b.total = max_table.max_num AND b.year = max_table.year ORDER BY b.year DESC) SELECT name, COUNT(year) FROM most_popular_male_name_by_year GROUP BY name ORDER BY count(year) DESC'  )
results = cursor.fetchall()
print(tabulate(results, headers=['Name', 'Count Top Male Name'], tablefmt="fancy_grid"))

╒═══════════╤═══════════════════════╕
│ Name      │   Count Top Male Name │
╞═══════════╪═══════════════════════╡
│ William   │                    10 │
├───────────┼───────────────────────┤
│ Mathias   │                     5 │
├───────────┼───────────────────────┤
│ Lucas     │                     5 │
├───────────┼───────────────────────┤
│ Kasper    │                     5 │
├───────────┼───────────────────────┤
│ Christian │                     4 │
├───────────┼───────────────────────┤
│ Frederik  │                     3 │
├───────────┼───────────────────────┤
│ Mikkel    │                     2 │
├───────────┼───────────────────────┤
│ Oscar     │                     1 │
├───────────┼───────────────────────┤
│ Noah      │                     1 │
├───────────┼───────────────────────┤
│ Martin    │                     1 │
├───────────┼───────────────────────┤
│ Alfred    │                     1 │
╘═══════════╧═══════════════════════╛


* The "Semi-classic" name, "William", has been number one for the largest number of years! By the way, if you happen to know Vinh_DK Youtube Channel (he is pretty famous among Vietnamese community in Denmark), this is his youngest son's name. 🥰

In [ ]:
# Commit changes and close connection
conn.commit()
conn.close()